In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Bancor Bonding Curve

> $price = m*supply^n$ formula. Parameterized by $supply$, $price$, and $balance$.

In [ ]:
#| default_exp pamm.bancor

In [ ]:
#| export
import param as pm
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv

In [ ]:
#| export
class BondingCurve(pm.Parameterized):
    supply = pm.Number(80, softbounds=(1, 100), bounds=(1, None), step=1)
    price = pm.Number(2, softbounds=(0.01, 5), bounds=(0.01, None),  step=0.01)
    balance = pm.Number(40, softbounds=(10, 500), bounds=(10, None), step=1)
    marketcap = pm.Number(4, constant=True)
    reserve_ratio = pm.Number(1, constant=True, step=0.01)
    n = pm.Number(constant=True)
    m = pm.Number(constant=True)
    
    mint_amount = pm.Number(1, step=0.1)
    deposit = pm.Number(constant=True)
    mint_price = pm.Number(constant=True)
    new_price = pm.Number(constant=True)
    
    mint = pm.Action(lambda self: self._mint())
    
    def __init__(self, **params):
        super().__init__(**params)
        self.update()
    
    @pm.depends('balance', 'supply', 'price', 'mint_amount', watch=True)
    def update(self):
        with pm.edit_constant(self):
            self.marketcap = self.price * self.supply
            self.reserve_ratio = self.balance / self.marketcap
            self.param['mint_amount'].bounds = [-self.supply+1, self.param['supply'].softbounds[1]-self.supply]
            self.n = ((1 / self.reserve_ratio) - 1)
            self.m = self.price / self.supply ** self.n
            if self.mint_amount == 0:
                self.deposit = 0
                self.mint_price = self.price
                self.new_price = self.price
            else:
                self.deposit = self.get_balance_deposit(self.mint_amount)
                self.mint_price = self.deposit / self.mint_amount
                self.new_price = self.get_price(self.supply+self.mint_amount)

    def get_price(self, supply):
        price = self.m * supply ** self.n
        return price
    
    def get_marketcap(self, supply):
        marketcap = self.get_price(supply) * supply
        return marketcap
    
    def get_balance(self, supply):
        balance = self.reserve_ratio * self.get_marketcap(supply)
        return balance
    
    # How much balance to deposit given a mint amount
    def get_balance_deposit(self, mint_amount):
        balance_deposit = self.get_balance(self.supply+mint_amount) - self.balance
        # balance_deposit = self.get_balance(self.supply) * ((mint_amount / self.supply + 1) ** (1 / self.reserve_ratio) - 1)
        return balance_deposit
            
    # How much balance to return given a burn amount
    def get_balance_return(self, burn_amount):
        balance_return = self.balance - self.get_balance(self.supply-burn_amount)
        return balance_return
        
    # How much supply minted given a balance deposit
    def get_mint_amount(self, balance_deposit):
        mint_amount = self.supply * ((balance_deposit / self.get_balance(self.supply) + 1) ** (self.reserve_ratio) - 1)
        return mint_amount
    
    # How much supply to burn given a desired balance return
    # def get_burn_amount(self, balance_return):
    #     pass
    
    def _mint(self):
        with pm.parameterized.discard_events(self):
            self.supply = self.supply + self.mint_amount
            self.balance = self.balance + self.deposit
            self.price = self.new_price
            self.mint_amount = 0
        self.param.trigger('supply', 'balance', 'price', 'mint_amount')
        
    def price_over_supply_curve(self):
        supply = np.linspace(*self.param['supply'].softbounds, num=1000)
        prices = self.get_price(supply)
        balance_slope = np.where(supply <= self.supply, prices, 0)
        marketcap_slope = np.where(supply <= self.supply, self.price, 0)
        future_supply = self.supply + self.mint_amount
        if future_supply > self.supply:
            balance_deposit = np.where((self.supply <= supply) & (supply <= future_supply), prices, 0)
        else:
            balance_deposit = np.where((future_supply <= supply) & (supply <= self.supply), prices, 0)
            
        new_price = np.where(supply <= future_supply, self.get_price(future_supply), 0)
        
        df = pd.DataFrame({
            'Supply': supply, 
            'Price': prices, 
            'Balance': balance_slope,
            'Marketcap': marketcap_slope,
            'Minting Deposit': balance_deposit,
            'New Price': new_price,
        })
        return df
    
    def view_price_over_supply_curve(self):
        price_over_supply_curve = self.price_over_supply_curve()
        price_curve = price_over_supply_curve.hvplot.line(x='Supply',y='Price')
        price_curve.opts( 
            color='purple', 
        )
        new_price = price_over_supply_curve.hvplot.area(x='Supply',y='New Price', y2='Marketcap')
        new_price.opts( 
            color='orange', 
            alpha=0.4,
        )
        balance_integral = price_over_supply_curve.hvplot.area(
            x='Supply', 
            y=['Marketcap', 'Balance', 'Minting Deposit'], 
            color=['green', 'blue', 'red'],
            alpha=0.4,
            stacked=False,
        )

        chart = price_curve * balance_integral * new_price
        return chart
    
    def view_points(self):
        current_price = (self.supply, self.price, 'Current Price')
        future_price = (self.supply+self.mint_amount, self.get_price(self.supply+self.mint_amount), 'Future Price')
        points = pd.DataFrame([future_price, current_price], columns=['x','y','label']).hvplot.scatter(
            x='x',
            y='y',
            by='label',
            color=['purple', 'orange'],
            size=80,) 
        return points
    
    @pm.depends('balance', 'supply', 'price', 'mint_amount')
    def view_chart(self):
        curve = self.view_price_over_supply_curve()
        points = self.view_points()
        chart = curve * points
        chart.opts(
            title="Bonding Curve Math",
            legend_position="top_right",
            xlim=self.param['supply'].softbounds,
            ylim=self.param['price'].softbounds,
            width=420,
            height=500,
        )
        return chart
    
    def view(self):
        return pn.Row(self, self.view_chart)

In [ ]:
b = BondingCurve()

In [ ]:
b.view()

Row
    [0] Column(margin=5, name='BondingCurve', width=300)
        [0] StaticText(value='<b>BondingCurve</b>')
        [1] FloatSlider(end=100, name='Supply', start=1, step=1, value=80)
        [2] FloatSlider(end=5, name='Price', start=0.01, step=0.01, value=2)
        [3] FloatSlider(end=500, name='Balance', start=10, step=1, value=40)
        [4] FloatInput(disabled=True, name='Marketcap', value=160)
        [5] FloatInput(disabled=True, name='Reserve ratio', step=0.01, value=0.25)
        [6] FloatInput(disabled=True, name='N', value=3.0)
        [7] FloatInput(disabled=True, name='M', value=3.90625e-06)
        [8] FloatSlider(end=20, name='Mint amount', start=-79, value=1)
        [9] FloatInput(disabled=True, name='Deposit', value=2.037813476562505)
        [10] FloatInput(disabled=True, name='Mint price', value=2.037813476562505)
        [11] FloatInput(disabled=True, name='New price', value=2.07594140625)
        [12] Button(name='Mint')
    [1] ParamMethod(method, _pane=HoloViews)

In [ ]:
b.price_over_supply_curve()

,Supply,Price,Balance,Marketcap,Minting Deposit,New Price
0,1.000000,0.000004,0.000004,2,0.0,2.450086
1,1.099099,0.000005,0.000005,2,0.0,2.450086
2,1.198198,0.000007,0.000007,2,0.0,2.450086
3,1.297297,0.000009,0.000009,2,0.0,2.450086
4,1.396396,0.000011,0.000011,2,0.0,2.450086
...,...,...,...,...,...,...
995,99.603604,3.859981,0.000000,0,0.0,0.000000
996,99.702703,3.871514,0.000000,0,0.0,0.000000
997,99.801802,3.883070,0.000000,0,0.0,0.000000
998,99.900901,3.894648,0.000000,0,0.0,0.000000


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()